## Libraries

In [2]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd
import Levenshtein

import warnings
warnings.filterwarnings('ignore')

## Extract Data

In [3]:
df_shopee = pd.read_excel('./DATA/Avoskin_Shopee.xlsx')
df_tokopedia = pd.read_excel('./DATA/Avoskin_Tokopedia.xlsx')
df_tiktok = pd.read_excel('./DATA/Avoskin_tiktok.xlsx')
df_website = pd.read_excel('./DATA/Avoskin_website.xlsx')
df_zalora = pd.read_excel('./DATA/Avoskin_Zalora.xlsx')
df_retail = pd.read_excel('./DATA/Avoskin_retail.xlsx')
df_gt = pd.read_excel('./DATA/Avoskin_GT.xlsx')

## Transform Data

- Data Shopee

In [4]:
df_shopee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 42 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   No. Pesanan                                                   500 non-null    object 
 1   Status Pesanan                                                500 non-null    object 
 2   No. Resi                                                      500 non-null    int64  
 3   Opsi Pengiriman                                               500 non-null    object 
 4   Antar ke counter/ pick-up                                     500 non-null    object 
 5   Pesanan Harus Dikirimkan Sebelum (Menghindari keterlambatan)  500 non-null    object 
 6   Waktu Pengiriman Diatur                                       499 non-null    object 
 7   Waktu Pesanan Dibuat                                          500 non-n

In [5]:
# Ambil kolom yang dibutuhkan dan tambahkan kolom baru
df_shopee_transformed = df_shopee.copy()

# Tambah kolom baru dan kosongi isinya
df_shopee_transformed['mapping_status'] = ''
df_shopee_transformed['mapping_product'] = ''
df_shopee_transformed['kategori'] = ''
df_shopee_transformed['total_harga_sebelum_diskon'] = df_shopee_transformed['Harga Awal'] * df_shopee_transformed['Jumlah']
df_shopee_transformed['total_harga_setelah_diskon'] = (
    (df_shopee_transformed['Harga Awal'] - df_shopee_transformed['Total Diskon'] 
    - df_shopee_transformed['Diskon Dari Penjual'] 
    - df_shopee_transformed['Diskon Dari Shopee']) * df_shopee_transformed['Jumlah']
)

# Kolom Username (Pembeli) diisi dengan 'Nama Penerima' + 'No. Telepon' jika null
df_shopee_transformed['mapping_pembeli'] = df_shopee_transformed['Username (Pembeli)']
df_shopee_transformed['mapping_pembeli'].fillna(df_shopee_transformed['Nama Penerima'] + ' ' + df_shopee_transformed['No. Telepon'], inplace=True)

# Tambah kolom dengan nilai tetap
df_shopee_transformed['channel'] = 'B2C'
df_shopee_transformed['brand'] = 'Avoskin'
df_shopee_transformed['store'] = 'Shopee'

# Atur urutan kolom seperti yang diminta
df_shopee_transformed = df_shopee_transformed.rename(columns={
    'No. Pesanan': 'no_pesanan',
    'Waktu Pesanan Dibuat': 'tanggal_pesanan_dibuat',
    'Status Pesanan': 'status',
    'SKU Induk': 'sku_induk',
    'Nama Produk': 'nama_produk',
    'Nama Variasi': 'nama_variasi',
    'Harga Awal': 'harga_awal',
    'Jumlah': 'jumlah',
    'Kota/Kabupaten': 'mapping_kota',
    'Provinsi': 'mapping_provinsi'
})[[ 
    'no_pesanan',
    'tanggal_pesanan_dibuat',
    'status',
    'mapping_status',
    'sku_induk',
    'nama_produk',
    'mapping_product',
    'nama_variasi',
    'kategori',
    'harga_awal',
    'jumlah',
    'total_harga_sebelum_diskon',
    'total_harga_setelah_diskon',
    'mapping_pembeli',
    'mapping_kota',
    'mapping_provinsi',
    'channel',
    'brand',
    'store'
]]


In [6]:
# Mengonversi kolom 'tanggal_pesanan_dibuat' menjadi tipe data datetime (format YYYY-MM-DD)
df_shopee_transformed['tanggal_pesanan_dibuat'] = pd.to_datetime(df_shopee_transformed['tanggal_pesanan_dibuat']).dt.date
df_shopee_transformed['tanggal_pesanan_dibuat'].head()

0    2021-12-01
1    2021-12-01
2    2021-12-01
3    2021-12-01
4    2021-12-01
Name: tanggal_pesanan_dibuat, dtype: object

In [7]:
df_shopee_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   no_pesanan                  500 non-null    object
 1   tanggal_pesanan_dibuat      500 non-null    object
 2   status                      500 non-null    object
 3   mapping_status              500 non-null    object
 4   sku_induk                   500 non-null    object
 5   nama_produk                 500 non-null    object
 6   mapping_product             500 non-null    object
 7   nama_variasi                6 non-null      object
 8   kategori                    500 non-null    object
 9   harga_awal                  500 non-null    int64 
 10  jumlah                      500 non-null    int64 
 11  total_harga_sebelum_diskon  500 non-null    int64 
 12  total_harga_setelah_diskon  500 non-null    int64 
 13  mapping_pembeli             500 non-null    object

- Tokopedia

In [8]:
df_tokopedia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 41 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Nomor                                     417 non-null    float64
 1   Nomor Invoice                             500 non-null    int64  
 2   Tanggal Pembayaran                        500 non-null    object 
 3   Status Terakhir                           500 non-null    object 
 4   Tanggal Pesanan Selesai                   497 non-null    object 
 5   Waktu Pesanan Selesai                     497 non-null    object 
 6   Tanggal Pesanan Dibatalkan                3 non-null      object 
 7   Waktu Pesanan Dibatalkan                  3 non-null      object 
 8   Nama Produk                               500 non-null    object 
 9   Tipe Produk                               500 non-null    object 
 10  Nomor SKU                             

In [9]:
# Ambil kolom yang dibutuhkan dan tambahkan kolom baru
df_tokopedia_transformed = df_tokopedia.copy()

# Tambah kolom baru dan kosongi isinya
df_tokopedia_transformed['mapping_status'] = ''
df_tokopedia_transformed['mapping_product'] = ''
df_tokopedia_transformed['kategori'] = ''
df_tokopedia_transformed['nama_variasi'] = ''

# Hitung total harga sebelum dan setelah diskon
df_tokopedia_transformed['total_harga_sebelum_diskon'] = df_tokopedia_transformed['Harga Awal (IDR)'] * df_tokopedia_transformed['Jumlah Produk Dibeli']
df_tokopedia_transformed['total_harga_setelah_diskon'] = (
    df_tokopedia_transformed['Harga Jual (IDR)'] - df_tokopedia_transformed['Jumlah Produk Dibeli']
)

# Kolom Username (Pembeli) diisi dengan 'Nama Penerima' + 'No. Telepon' jika null
df_tokopedia_transformed['mapping_pembeli'] = df_tokopedia_transformed['Nama Pembeli']
df_tokopedia_transformed['mapping_pembeli'].fillna(df_tokopedia_transformed['Nama Penerima'] + ' ' + df_tokopedia_transformed['No Telp Pembeli'], inplace=True)

# Tambah kolom dengan nilai tetap
df_tokopedia_transformed['channel'] = 'B2C'
df_tokopedia_transformed['brand'] = 'Avoskin'
df_tokopedia_transformed['store'] = 'Tokopedia'

# Mengubah nama kolom sesuai dengan yang diminta
df_tokopedia_transformed = df_tokopedia_transformed.rename(columns={
    'Nomor Invoice': 'no_pesanan',
    'Tanggal Pembayaran': 'tanggal_pesanan_dibuat',
    'Status Terakhir': 'status',
    'Nomor SKU': 'sku_induk',
    'Nama Produk': 'nama_produk',
    'nama_variasi': 'nama_variasi',
    'Harga Awal (IDR)': 'harga_awal',
    'Jumlah Produk Dibeli': 'jumlah',
    'Kota': 'mapping_kota',
    'Provinsi': 'mapping_provinsi'
})

# Mengatur urutan kolom sesuai dengan yang diminta
df_tokopedia_transformed = df_tokopedia_transformed[[
    'no_pesanan', 
    'tanggal_pesanan_dibuat', 
    'status', 
    'mapping_status', 
    'sku_induk', 
    'nama_produk', 
    'mapping_product', 
    'nama_variasi', 
    'kategori', 
    'harga_awal', 
    'jumlah', 
    'total_harga_sebelum_diskon', 
    'total_harga_setelah_diskon', 
    'mapping_pembeli', 
    'mapping_kota', 
    'mapping_provinsi', 
    'channel', 
    'brand', 
    'store'
]]


In [10]:
# Mengonversi kolom 'tanggal_pesanan_dibuat' menjadi tipe data datetime (format YYYY-MM-DD)
df_tokopedia_transformed['tanggal_pesanan_dibuat'] = pd.to_datetime(df_tokopedia_transformed['tanggal_pesanan_dibuat']).dt.date
df_tokopedia_transformed['tanggal_pesanan_dibuat'].head()

0    2021-01-12
1    2021-01-12
2    2021-01-12
3    2021-01-12
4    2021-01-12
Name: tanggal_pesanan_dibuat, dtype: object

In [11]:
df_tokopedia_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   no_pesanan                  500 non-null    int64 
 1   tanggal_pesanan_dibuat      500 non-null    object
 2   status                      500 non-null    object
 3   mapping_status              500 non-null    object
 4   sku_induk                   500 non-null    int64 
 5   nama_produk                 500 non-null    object
 6   mapping_product             500 non-null    object
 7   nama_variasi                500 non-null    object
 8   kategori                    500 non-null    object
 9   harga_awal                  500 non-null    int64 
 10  jumlah                      500 non-null    int64 
 11  total_harga_sebelum_diskon  500 non-null    int64 
 12  total_harga_setelah_diskon  500 non-null    int64 
 13  mapping_pembeli             500 non-null    object

- Data Tiktok

In [12]:
df_tiktok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 55 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Order ID                        149 non-null    int64  
 1   Order Status                    149 non-null    object 
 2   Order Substatus                 149 non-null    object 
 3   Cancelation/Return Type         5 non-null      object 
 4   Normal or Pre-order             149 non-null    object 
 5   SKU ID                          149 non-null    int64  
 6   Seller SKU                      145 non-null    float64
 7   Product Name                    149 non-null    object 
 8   Variation                       149 non-null    object 
 9   Quantity                        149 non-null    int64  
 10  Sku Quantity of return          149 non-null    int64  
 11  SKU Unit Original Price         149 non-null    object 
 12  SKU Subtotal Before Discount    149 

In [13]:
# Ambil kolom yang dibutuhkan dan tambahkan kolom baru
df_tiktok_transformed = df_tiktok.copy()

# Tambah kolom baru dan kosongi isinya
df_tiktok_transformed['mapping_status'] = ''
df_tiktok_transformed['mapping_product'] = ''
df_tiktok_transformed['kategori'] = ''

df_tiktok_transformed['SKU Unit Original Price'] = df_tiktok_transformed['SKU Unit Original Price'].str.replace('IDR', '', regex=False).str.replace('.', '', regex=False).str.strip()
df_tiktok_transformed['SKU Unit Original Price'] = df_tiktok_transformed['SKU Unit Original Price'].fillna(0)
df_tiktok_transformed['SKU Unit Original Price'] = df_tiktok_transformed['SKU Unit Original Price'].astype(int)
df_tiktok_transformed['SKU Subtotal After Discount'] = df_tiktok_transformed['SKU Subtotal After Discount'].str.replace('IDR', '', regex=False).str.replace('.', '', regex=False).str.strip()
df_tiktok_transformed['SKU Subtotal After Discount'] = df_tiktok_transformed['SKU Subtotal After Discount'].fillna(0)
df_tiktok_transformed['SKU Subtotal After Discount'] = df_tiktok_transformed['SKU Subtotal After Discount'].astype(int)
df_tiktok_transformed['SKU Subtotal Before Discount'] = df_tiktok_transformed['SKU Subtotal Before Discount'].str.replace('IDR', '', regex=False).str.replace('.', '', regex=False).str.strip()
df_tiktok_transformed['SKU Subtotal Before Discount'] = df_tiktok_transformed['SKU Subtotal Before Discount'].fillna(0)
df_tiktok_transformed['SKU Subtotal Before Discount'] = df_tiktok_transformed['SKU Subtotal Before Discount'].astype(int)

# Hitung total harga sebelum dan setelah diskon
df_tiktok_transformed['total_harga_sebelum_diskon'] = df_tiktok_transformed['SKU Subtotal Before Discount']
df_tiktok_transformed['total_harga_setelah_diskon'] = df_tiktok_transformed['SKU Subtotal After Discount']

# Kolom Username (Pembeli) diisi dengan 'Nama Penerima' + 'No. Telepon' jika null
df_tiktok_transformed['mapping_pembeli'] = df_tiktok_transformed['Buyer Username']
df_tiktok_transformed['mapping_pembeli'].fillna(df_tiktok_transformed['Recipient'] + ' ' + df_tiktok_transformed['Phone #'], inplace=True)

# Tambah kolom dengan nilai tetap
df_tiktok_transformed['channel'] = 'B2C'
df_tiktok_transformed['brand'] = 'Avoskin'
df_tiktok_transformed['store'] = 'TikTok'

# Mengatur urutan kolom
df_tiktok_transformed = df_tiktok_transformed[[
    'Order ID', 
    'Created Time', 
    'Order Status', 
    'mapping_status', 
    'Seller SKU', 
    'Product Name', 
    'mapping_product', 
    'Variation', 
    'kategori', 
    'SKU Unit Original Price', 
    'Quantity', 
    'total_harga_sebelum_diskon', 
    'total_harga_setelah_diskon', 
    'mapping_pembeli', 
    'Regency and City', 
    'Province', 
    'channel', 
    'brand', 
    'store'
]]

# Mengganti nama kolom
df_tiktok_transformed.columns = [
    'no_pesanan',
    'tanggal_pesanan_dibuat',
    'status',
    'mapping_status',
    'sku_induk',
    'nama_produk',
    'mapping_product',
    'nama_variasi',
    'kategori',
    'harga_awal',
    'jumlah',
    'total_harga_sebelum_diskon',
    'total_harga_setelah_diskon',
    'mapping_pembeli',
    'mapping_kota',
    'mapping_provinsi',
    'channel',
    'brand',
    'store'
]

In [14]:
# Mengonversi kolom 'tanggal_pesanan_dibuat' menjadi tipe data datetime (format YYYY-MM-DD)
df_tiktok_transformed['tanggal_pesanan_dibuat'] = pd.to_datetime(df_tiktok_transformed['tanggal_pesanan_dibuat']).dt.date
df_tiktok_transformed['tanggal_pesanan_dibuat'].head()

0    2021-12-31
1    2021-12-31
2    2021-12-30
3    2021-12-30
4    2021-12-30
Name: tanggal_pesanan_dibuat, dtype: object

In [15]:
df_tiktok_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   no_pesanan                  149 non-null    int64  
 1   tanggal_pesanan_dibuat      149 non-null    object 
 2   status                      149 non-null    object 
 3   mapping_status              149 non-null    object 
 4   sku_induk                   145 non-null    float64
 5   nama_produk                 149 non-null    object 
 6   mapping_product             149 non-null    object 
 7   nama_variasi                149 non-null    object 
 8   kategori                    149 non-null    object 
 9   harga_awal                  149 non-null    int32  
 10  jumlah                      149 non-null    int64  
 11  total_harga_sebelum_diskon  149 non-null    int32  
 12  total_harga_setelah_diskon  149 non-null    int32  
 13  mapping_pembeli             149 non

- Data Website

In [16]:
df_website.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Order ID                292 non-null    object        
 1   Email                   292 non-null    object        
 2   Tanggal Order           292 non-null    datetime64[ns]
 3   Order Status            292 non-null    object        
 4   Nama Customer           292 non-null    object        
 5   Phone                   292 non-null    int64         
 6   Province                292 non-null    object        
 7   City                    292 non-null    object        
 8   District                0 non-null      float64       
 9   Postcode                292 non-null    int64         
 10  Payment Method          292 non-null    object        
 11  Shipping Method         292 non-null    object        
 12  AWB                     226 non-null    object    

In [17]:
# Ambil kolom yang dibutuhkan dan tambahkan kolom baru
df_website_transformed = df_website.copy()

# Tambah kolom baru dan kosongi isinya
df_website_transformed['mapping_status'] = ''
df_website_transformed['mapping_product'] = ''
df_website_transformed['kategori'] = ''
df_website_transformed['nama_variasi'] = ''

# Hitung total harga sebelum dan setelah diskon
df_website_transformed['total_harga_sebelum_diskon'] = df_website_transformed['Harga Awal'] * df_website_transformed['Qty']
df_website_transformed['total_harga_setelah_diskon'] =  df_website_transformed['Subtotal']

# Kolom Username (Pembeli) diisi dengan 'Nama Penerima' + 'No. Telepon' jika null
df_website_transformed['mapping_pembeli'] = df_website_transformed['Nama Customer']
df_website_transformed['mapping_pembeli'].fillna(df_website_transformed['Phone'], inplace=True)

# Tambah kolom dengan nilai tetap
df_website_transformed['channel'] = 'B2C'
df_website_transformed['brand'] = 'Avoskin'
df_website_transformed['store'] = 'Website'

# Atur urutan kolom seperti yang diminta
df_website_transformed = df_website_transformed[[
    'Order ID', 
    'Tanggal Order', 
    'Order Status', 
    'mapping_status', 
    'GS1 Product', 
    'Nama Product', 
    'mapping_product', 
    'nama_variasi', 
    'kategori', 
    'Harga Awal', 
    'Qty', 
    'total_harga_sebelum_diskon', 
    'total_harga_setelah_diskon', 
    'mapping_pembeli', 
    'City', 
    'Province', 
    'channel', 
    'brand', 
    'store'
]]

# Ganti nama kolom
df_website_transformed.columns = [
    'no_pesanan',
    'tanggal_pesanan_dibuat',
    'status',
    'mapping_status',
    'sku_induk',
    'nama_produk',
    'mapping_product',
    'nama_variasi',
    'kategori',
    'harga_awal',
    'jumlah',
    'total_harga_sebelum_diskon',
    'total_harga_setelah_diskon',
    'mapping_pembeli',
    'mapping_kota',
    'mapping_provinsi',
    'channel',
    'brand',
    'store'
]

In [18]:
# Mengonversi kolom 'tanggal_pesanan_dibuat' menjadi tipe data datetime (format YYYY-MM-DD)
df_website_transformed['tanggal_pesanan_dibuat'] = pd.to_datetime(df_website_transformed['tanggal_pesanan_dibuat']).dt.date
df_website_transformed['tanggal_pesanan_dibuat'].head()

0    2021-12-01
1    2021-12-01
2    2021-12-01
3    2021-12-01
4    2021-12-01
Name: tanggal_pesanan_dibuat, dtype: object

In [19]:
df_website_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   no_pesanan                  292 non-null    object
 1   tanggal_pesanan_dibuat      292 non-null    object
 2   status                      292 non-null    object
 3   mapping_status              292 non-null    object
 4   sku_induk                   211 non-null    object
 5   nama_produk                 292 non-null    object
 6   mapping_product             292 non-null    object
 7   nama_variasi                292 non-null    object
 8   kategori                    292 non-null    object
 9   harga_awal                  292 non-null    int64 
 10  jumlah                      292 non-null    int64 
 11  total_harga_sebelum_diskon  292 non-null    int64 
 12  total_harga_setelah_diskon  292 non-null    int64 
 13  mapping_pembeli             292 non-null    object

- Data Zalora

In [20]:
df_zalora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 71 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Order Item Id                 386 non-null    int64  
 1   Zalora Id                     386 non-null    int64  
 2   Seller SKU                    386 non-null    object 
 3   Zalora SKU                    386 non-null    object 
 4   Created at                    386 non-null    object 
 5   Updated at                    386 non-null    object 
 6   Order Number                  386 non-null    int64  
 7   Order Source                  386 non-null    object 
 8   Order Currency                386 non-null    object 
 9   Customer Name                 386 non-null    object 
 10  National Registration Number  0 non-null      float64
 11  Shipping Name                 386 non-null    object 
 12  Shipping Address              386 non-null    object 
 13  Shipp

In [21]:
# Ambil kolom yang dibutuhkan dan tambahkan kolom baru
df_zalora_transformed = df_zalora.copy()

# Tambah kolom baru dan kosongi isinya
df_zalora_transformed['mapping_status'] = ''
df_zalora_transformed['mapping_product'] = ''
df_zalora_transformed['kategori'] = ''
df_zalora_transformed['City'] = ''
df_zalora_transformed['Province'] = ''
df_zalora_transformed['Jumlah'] = 1

# Hitung total harga sebelum dan setelah diskon
df_zalora_transformed['total_harga_sebelum_diskon'] = df_zalora_transformed['Unit Price'] * 1
df_zalora_transformed['total_harga_setelah_diskon'] = df_zalora_transformed['Paid Price'] * 1

# Kolom Username (Pembeli) diisi dengan 'Nama Penerima' + 'No. Telepon' jika null
df_zalora_transformed['mapping_pembeli'] = df_zalora_transformed['Customer Name']
df_zalora_transformed['mapping_pembeli'].fillna(df_zalora_transformed['Shipping Name']+ ' ' + df_zalora_transformed['Billing Phone Number2'], inplace=True)

# Tambah kolom dengan nilai tetap
df_zalora_transformed['channel'] = 'B2C'
df_zalora_transformed['brand'] = 'Avoskin'
df_zalora_transformed['store'] = 'Zalora'

# Atur urutan kolom seperti yang diminta
df_zalora_transformed = df_zalora_transformed[[
    'Order Number', 
    'Created at', 
    'Status', 
    'mapping_status', 
    'Seller SKU', 
    'Item Name', 
    'mapping_product', 
    'Variation', 
    'kategori', 
    'Unit Price', 
    'Jumlah', 
    'total_harga_sebelum_diskon', 
    'total_harga_setelah_diskon', 
    'mapping_pembeli', 
    'City', 
    'Province', 
    'channel', 
    'brand', 
    'store'
]]

# Mengganti nama kolom
df_zalora_transformed.columns = [
    'no_pesanan',
    'tanggal_pesanan_dibuat',
    'status',
    'mapping_status',
    'sku_induk',
    'nama_produk',
    'mapping_product',
    'nama_variasi',
    'kategori',
    'harga_awal',
    'jumlah',
    'total_harga_sebelum_diskon',
    'total_harga_setelah_diskon',
    'mapping_pembeli',
    'mapping_kota',
    'mapping_provinsi',
    'channel',
    'brand',
    'store'
]

In [22]:
# Mengonversi kolom 'tanggal_pesanan_dibuat' menjadi tipe data datetime (format YYYY-MM-DD)
df_zalora_transformed['tanggal_pesanan_dibuat'] = pd.to_datetime(df_zalora_transformed['tanggal_pesanan_dibuat']).dt.date
df_zalora_transformed['tanggal_pesanan_dibuat'].head()

0    2021-12-30
1    2021-12-30
2    2021-12-30
3    2021-12-30
4    2021-12-30
Name: tanggal_pesanan_dibuat, dtype: object

In [23]:
df_zalora_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   no_pesanan                  386 non-null    int64 
 1   tanggal_pesanan_dibuat      386 non-null    object
 2   status                      386 non-null    object
 3   mapping_status              386 non-null    object
 4   sku_induk                   386 non-null    object
 5   nama_produk                 386 non-null    object
 6   mapping_product             386 non-null    object
 7   nama_variasi                386 non-null    object
 8   kategori                    386 non-null    object
 9   harga_awal                  386 non-null    int64 
 10  jumlah                      386 non-null    int64 
 11  total_harga_sebelum_diskon  386 non-null    int64 
 12  total_harga_setelah_diskon  386 non-null    int64 
 13  mapping_pembeli             386 non-null    object

- Data Retail

In [24]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Tanggal Pesanan       501 non-null    datetime64[ns]
 1   Username              501 non-null    object        
 2   No Pesanan            501 non-null    int64         
 3   SKU Induk             483 non-null    float64       
 4   Nama Produk           501 non-null    object        
 5   Nama Variasi          0 non-null      float64       
 6   Jumlah Pesanan (Qty)  501 non-null    int64         
 7   Harga Awal            501 non-null    int64         
 8   Total Diskon          501 non-null    int64         
 9   Kota                  495 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 39.3+ KB


In [25]:
# Salin DataFrame dan tambahkan kolom baru
df_retail_transformed = df_retail.copy()

# Tambah kolom baru dan kosongi isinya
df_retail_transformed['mapping_status'] = ''
df_retail_transformed['mapping_product'] = ''
df_retail_transformed['kategori'] = ''
df_retail_transformed['Province'] = ''
df_retail_transformed['Status'] = 'Selesai'

# Hitung total harga sebelum dan setelah diskon
df_retail_transformed['total_harga_sebelum_diskon'] = df_retail_transformed['Harga Awal'] * df_retail_transformed['Jumlah Pesanan (Qty)']
df_retail_transformed['total_harga_setelah_diskon'] = df_retail_transformed['total_harga_sebelum_diskon'] - df_retail_transformed['Total Diskon']

# Kolom Username (Pembeli) diisi dengan 'Nama Penerima' + 'No. Telepon' jika null
df_retail_transformed['mapping_pembeli'] = df_retail_transformed['Username']

# Tambah kolom dengan nilai tetap
df_retail_transformed['channel'] = 'B2B'
df_retail_transformed['brand'] = 'Avoskin'
df_retail_transformed['store'] = df_retail_transformed['Username']

# Atur urutan kolom seperti yang diminta
df_retail_transformed = df_retail_transformed.rename(columns={
    'No Pesanan': 'no_pesanan',
    'Tanggal Pesanan': 'tanggal_pesanan_dibuat',
    'Status': 'status',
    'mapping_status': 'mapping_status',
    'SKU Induk': 'sku_induk',
    'Nama Produk': 'nama_produk',
    'mapping_product': 'mapping_product',
    'Nama Variasi': 'nama_variasi',
    'kategori': 'kategori',
    'Harga Awal': 'harga_awal',
    'Jumlah Pesanan (Qty)': 'jumlah',
    'total_harga_sebelum_diskon': 'total_harga_sebelum_diskon',
    'total_harga_setelah_diskon': 'total_harga_setelah_diskon',
    'mapping_pembeli': 'mapping_pembeli',
    'Kota': 'mapping_kota',
    'Province': 'mapping_provinsi',
    'channel': 'channel',
    'brand': 'brand',
    'store': 'store'
})

# Atur urutan kolom sesuai yang diminta
df_retail_transformed = df_retail_transformed[[
    'no_pesanan', 
    'tanggal_pesanan_dibuat', 
    'status', 
    'mapping_status', 
    'sku_induk', 
    'nama_produk', 
    'mapping_product', 
    'nama_variasi', 
    'kategori', 
    'harga_awal', 
    'jumlah', 
    'total_harga_sebelum_diskon', 
    'total_harga_setelah_diskon', 
    'mapping_pembeli', 
    'mapping_kota', 
    'mapping_provinsi', 
    'channel', 
    'brand', 
    'store'
]]

In [26]:
# Mengonversi kolom 'tanggal_pesanan_dibuat' menjadi tipe data datetime (format YYYY-MM-DD)
df_retail_transformed['tanggal_pesanan_dibuat'] = pd.to_datetime(df_retail_transformed['tanggal_pesanan_dibuat']).dt.date
df_retail_transformed['tanggal_pesanan_dibuat'].head()

0    2021-12-01
1    2021-12-01
2    2021-12-01
3    2021-12-01
4    2021-12-01
Name: tanggal_pesanan_dibuat, dtype: object

In [27]:
df_retail_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   no_pesanan                  501 non-null    int64  
 1   tanggal_pesanan_dibuat      501 non-null    object 
 2   status                      501 non-null    object 
 3   mapping_status              501 non-null    object 
 4   sku_induk                   483 non-null    float64
 5   nama_produk                 501 non-null    object 
 6   mapping_product             501 non-null    object 
 7   nama_variasi                0 non-null      float64
 8   kategori                    501 non-null    object 
 9   harga_awal                  501 non-null    int64  
 10  jumlah                      501 non-null    int64  
 11  total_harga_sebelum_diskon  501 non-null    int64  
 12  total_harga_setelah_diskon  501 non-null    int64  
 13  mapping_pembeli             501 non

- Data Modern Trade (GT)

In [28]:
df_gt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Tanggal Pesanan       501 non-null    datetime64[ns]
 1   Channel               501 non-null    object        
 2   Username              501 non-null    object        
 3   No Pesanan            501 non-null    object        
 4   SKU Induk             493 non-null    float64       
 5   Nama Produk           501 non-null    object        
 6   Nama Variasi          0 non-null      float64       
 7   Jumlah Pesanan (Qty)  501 non-null    int64         
 8   Harga Awal            501 non-null    int64         
 9   Total Diskon          501 non-null    int64         
 10  Kota                  501 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 43.2+ KB


In [29]:
# Salin DataFrame dan tambahkan kolom baru
df_gt_transformed = df_gt.copy()

# Tambah kolom baru dan kosongi isinya
df_gt_transformed['mapping_status'] = ''
df_gt_transformed['mapping_product'] = ''
df_gt_transformed['kategori'] = ''
df_gt_transformed['Province'] = ''
df_gt_transformed['Status'] = 'Selesai'
df_gt_transformed['Total Diskon'] = df_gt_transformed['Total Diskon'].astype(int)

# Hitung total harga sebelum dan setelah diskon
df_gt_transformed['total_harga_sebelum_diskon'] = df_gt_transformed['Harga Awal'] * df_gt_transformed['Jumlah Pesanan (Qty)']
df_gt_transformed['total_harga_setelah_diskon'] = df_gt_transformed['total_harga_sebelum_diskon'] - df_gt_transformed['Total Diskon']

# Kolom Username (Pembeli) diisi dengan 'Nama Penerima' + 'No. Telepon' jika null
df_gt_transformed['mapping_pembeli'] = df_gt_transformed['Username']

# Tambah kolom dengan nilai tetap
df_gt_transformed['channel'] = 'B2B'
df_gt_transformed['brand'] = 'Avoskin'
df_gt_transformed['store'] = df_gt_transformed['Username']

# Atur urutan kolom seperti yang diminta
# Define the new column names
new_column_names = {
    'No Pesanan': 'no_pesanan',
    'Tanggal Pesanan': 'tanggal_pesanan_dibuat',
    'Status': 'status',
    'mapping_status': 'mapping_status',
    'SKU Induk': 'sku_induk',
    'Nama Produk': 'nama_produk',
    'mapping_product': 'mapping_product',
    'Nama Variasi': 'nama_variasi',
    'kategori': 'kategori',
    'Harga Awal': 'harga_awal',
    'Jumlah Pesanan (Qty)': 'jumlah',
    'total_harga_sebelum_diskon': 'total_harga_sebelum_diskon',
    'total_harga_setelah_diskon': 'total_harga_setelah_diskon',
    'mapping_pembeli': 'mapping_pembeli',
    'Kota': 'mapping_kota',
    'Province': 'mapping_provinsi',
    'channel': 'channel',
    'brand': 'brand',
    'store': 'store'
}

# Rename columns
df_gt_transformed = df_gt_transformed.rename(columns=new_column_names)

# Reorder columns
df_gt_transformed = df_gt_transformed[[
    'no_pesanan',
    'tanggal_pesanan_dibuat',
    'status',
    'mapping_status',
    'sku_induk',
    'nama_produk',
    'mapping_product',
    'nama_variasi',
    'kategori',
    'harga_awal',
    'jumlah',
    'total_harga_sebelum_diskon',
    'total_harga_setelah_diskon',
    'mapping_pembeli',
    'mapping_kota',
    'mapping_provinsi',
    'channel',
    'brand',
    'store'
]]

In [30]:
# Mengonversi kolom 'tanggal_pesanan_dibuat' menjadi tipe data datetime (format YYYY-MM-DD)
df_gt_transformed['tanggal_pesanan_dibuat'] = pd.to_datetime(df_gt_transformed['tanggal_pesanan_dibuat']).dt.date
df_gt_transformed['tanggal_pesanan_dibuat'].head()

0    2021-12-17
1    2021-12-17
2    2021-12-17
3    2021-12-17
4    2021-12-17
Name: tanggal_pesanan_dibuat, dtype: object

In [31]:
df_gt_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   no_pesanan                  501 non-null    object 
 1   tanggal_pesanan_dibuat      501 non-null    object 
 2   status                      501 non-null    object 
 3   mapping_status              501 non-null    object 
 4   sku_induk                   493 non-null    float64
 5   nama_produk                 501 non-null    object 
 6   mapping_product             501 non-null    object 
 7   nama_variasi                0 non-null      float64
 8   kategori                    501 non-null    object 
 9   harga_awal                  501 non-null    int64  
 10  jumlah                      501 non-null    int64  
 11  total_harga_sebelum_diskon  501 non-null    int64  
 12  total_harga_setelah_diskon  501 non-null    int64  
 13  mapping_pembeli             501 non

- Combine Data

In [32]:
combined_df = pd.concat([df_shopee_transformed, 
                         df_tokopedia_transformed,
                         df_tiktok_transformed,
                         df_website_transformed,
                         df_zalora_transformed,
                         df_retail_transformed,
                         df_gt_transformed], ignore_index=True)

- Merging Data

In [33]:
reference_status = pd.read_excel('./DATA/Technical Test Skill.xlsx', sheet_name='database status pesanan')
reference_product_kategori = pd.read_excel('./DATA/Technical Test Skill.xlsx', sheet_name='database Nama Product dan Kateg')
reference_region = pd.read_excel('./DATA/Technical Test Skill.xlsx', sheet_name='database region')

In [34]:
# Fungsi untuk menemukan nama produk paling mirip
def find_closest_match_l(input_text, reference_list):
    closest_match = ''
    lowest_distance = float('inf')
    
    input_text = input_text.lower()
    
    for ref in reference_list:
        distance = Levenshtein.distance(input_text, ref.lower())
        
        if distance < lowest_distance:
            lowest_distance = distance
            closest_match = ref
    
    return closest_match

refrence_name = reference_product_kategori['product']
combined_df['mapping_product'] = combined_df['nama_produk'].apply(lambda x: find_closest_match_l(x, refrence_name))

In [35]:
combined_df.head(3)

,no_pesanan,tanggal_pesanan_dibuat,status,mapping_status,sku_induk,nama_produk,mapping_product,nama_variasi,kategori,harga_awal,jumlah,total_harga_sebelum_diskon,total_harga_setelah_diskon,mapping_pembeli,mapping_kota,mapping_provinsi,channel,brand,store
0,010T1VTVHY,2021-12-01,Selesai,,4006,Avoskin Perfect Hydrating Treatment Essence 30...,Perfect Hydrating Treatment Essence (30 ml),NaN,,172852,1,172852,144852,MOGwOwsx,KAB. SLEMAN,DI YOGYAKARTA,B2C,Avoskin,Shopee
1,010EM4E0R6,2021-12-01,Selesai,,1122,BLP X Avoskin Multipurpose Tinted Sunscreen 5 ...,BLP x Avoskin Multipurpose Tinted Sunscreen Li...,Light 5 gr,,135624,1,135624,123624,uzlVoaIk,KOTA SEMARANG,JAWA TENGAH,B2C,Avoskin,Shopee
2,011730NWSA,2021-12-01,Selesai,,0021,Avoskin Your Skin Bae Toner Salicylic Acid 1% ...,YSB Toner Salicylic Acid 1% + Zinc + Tea Tree ...,NaN,,174730,31,5416630,5416630,JUlUCEOm,KOTA SURAKARTA (SOLO),JAWA TENGAH,B2C,Avoskin,Shopee


In [36]:
# Merge combined_df dengan reference_status berdasarkan kolom status
merged_df = pd.merge(combined_df, reference_status, how='left', left_on='status', right_on='status')
combined_df['mapping_status'] = merged_df['mapping_status_y']
merged_df = pd.merge(combined_df, reference_product_kategori, how='left', left_on='mapping_product', right_on='product')
combined_df['kategori'] = merged_df['kategori_y']
merged_df = pd.merge(combined_df, reference_region, how='left', left_on='mapping_kota', right_on='kota')
combined_df['mapping_kota'] = merged_df['mapping_kota_y']
combined_df['mapping_provinsi'] = merged_df['mapping_provinsi_y']

combined_df['nama_variasi'] = combined_df['nama_variasi'].apply(lambda x: '' if pd.isna(x) or x == '' else x)
combined_df['mapping_kota'] = combined_df['mapping_kota'].apply(lambda x: 'Tidak Diketahui' if pd.isna(x) or x == '' else x)
combined_df['mapping_provinsi'] = combined_df['mapping_provinsi'].apply(lambda x: 'Tidak Diketahui' if pd.isna(x) or x == '' else x)


In [40]:
combined_df['tanggal_pesanan_dibuat'].unique()

array([datetime.date(2021, 12, 1), datetime.date(2021, 12, 2),
       datetime.date(2021, 12, 3), datetime.date(2021, 1, 12),
       datetime.date(2021, 12, 31), datetime.date(2021, 12, 30),
       datetime.date(2021, 12, 29), datetime.date(2021, 12, 28),
       datetime.date(2021, 12, 26), datetime.date(2021, 12, 25),
       datetime.date(2021, 12, 24), datetime.date(2021, 12, 23),
       datetime.date(2021, 12, 22), datetime.date(2021, 12, 21),
       datetime.date(2021, 12, 17), datetime.date(2021, 12, 16),
       datetime.date(2021, 12, 15), datetime.date(2021, 12, 14),
       datetime.date(2021, 12, 13), datetime.date(2021, 12, 12),
       datetime.date(2021, 12, 11), datetime.date(2021, 12, 10),
       datetime.date(2021, 12, 9), datetime.date(2021, 12, 8),
       datetime.date(2021, 12, 7), datetime.date(2021, 12, 6),
       datetime.date(2021, 12, 5), datetime.date(2021, 12, 4),
       datetime.date(2021, 12, 18), datetime.date(2021, 12, 19),
       datetime.date(2021, 12, 20),

In [39]:
# Load credentials
with open('google_service_account.json', 'rb') as file:
    key = json.load(file)

# Define the scope for Google Drive and Sheets API
scope = ['https://www.googleapis.com/auth/drive', 'https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_dict(key, scope)
client = gspread.authorize(creds)

# Open the spreadsheet using its ID (which is extracted from the URL)
spreadsheet_id = '1vlihBL_lznMnMsX-_kElV5X3tA4BZUH4C2p9XRYuynI'  # The ID part from the URL
sheet = client.open_by_key(spreadsheet_id)

# Access the worksheet by its name
export = sheet.worksheet('DATA')

# Update the worksheet with new data from 'combined_df'
export.update([combined_df.columns.values.tolist()] + combined_df.astype(str).values.tolist())

{'spreadsheetId': '1vlihBL_lznMnMsX-_kElV5X3tA4BZUH4C2p9XRYuynI',
 'updatedRange': 'DATA!A1:S2830',
 'updatedRows': 2830,
 'updatedColumns': 19,
 'updatedCells': 53770}